In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [ ]:
# Importing the necessary libraries
import figs.render.capture_generation as pg
import figs.visualize.plot_trajectories as pt
import figs.visualize.generate_videos as gv

from figs.simulator import Simulator
from figs.control.vehicle_rate_mpc import VehicleRateMPC

In [3]:
# FiGS Capture Examples (scene_name, capture_name)
capture_examples = [
    'backroom'
]

# FiGS Simulate Examples (scene_name, rollout_name, frame_name, policy_name, course_name)
simulate_examples = [
    ('flightroom', 'baseline', 'carl', 'vrmpc_fr', 'extended_traj_track'),
    ('backroom',   'baseline', 'carl', 'vrmpc_fr', 'cluttered_env_track'),
    ('mid_gate',   'baseline', 'carl', 'vrmpc_fr', 'robustness_track'),
    ('src_open',   'baseline', 'carl', 'vrmpc_fr', 'infinity')
]

In [ ]:
# Generate the FiGS environment
for scene_name in capture_examples:
    print("=============================================================")
    print(f"Generating GSplat for [{scene_name}]")
    print("-------------------------------------------------------------")

    pg.generate_gsplat(scene_name)

In [ ]:
# Simulate within the FiGS environment
for scene, rollout, frame, policy, course in simulate_examples:
    print("=============================================================")
    print(f"Simulating {scene} scene with {course} course")
    print("-------------------------------------------------------------")

    # Load the policy and simulator
    sim = Simulator(scene,rollout,frame)
    ctl = VehicleRateMPC(course,policy,frame)

    # Use the ideal trajectory in VehicleRateMPC to get initial conditions and final time
    t0,tf,x0 = ctl.tXUd[0,0],ctl.tXUd[0,-1],ctl.tXUd[1:11,0]

    # Simulate the policy
    Tro,Xro,Uro,Imgs,_,_ = sim.simulate(ctl,t0,tf,x0)

    # Output the results
    gv.images_to_mp4(Imgs,course+'_'+scene+'.mp4', ctl.hz)       # Save the video
    pt.plot_RO_spatial((Tro,Xro,Uro))                               # Plot the spatial trajectory

    # Clear the memory of the policy to avoid improper re-initialization of ACADOS
    del ctl